In [20]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, sosfiltfilt
import h5py
import matplotlib
from scipy.ndimage import uniform_filter1d
from scipy.signal import spectrogram
from scipy.signal.windows import hamming
from scipy.signal import correlate

%matplotlib tk

In [21]:
'''
Parameter Setup
'''
vmines, vmaxes = -1, 1
vminkf, vmaxfk = 20, 80 # Adjusting the colormap scale

In [22]:
'''
Read Dataset
'''
filename = "onyxAcquisition_2025-11-07_19.12.59_UTC_000060.h5"
data_path = "/Acquisition/Raw[0]/RawData"

print("Reading data...")
try:
    with h5py.File(filename, "r") as f:
        X = f[data_path][:]
        fs = f["/Acquisition"].attrs["PulseRate"]
        dx = f["/Acquisition"].attrs["SpatialSamplingInterval"]
        num_locs = f["/Acquisition"].attrs["NumberOfLoci"]

        print("Metadata is successful:")
        print(f" -> fs = {fs:.1f} Hz")
        print(f" -> dx = {dx:.3f} m")
        print(f" -> num_locs = {num_locs} Channels")

        # Leer datos (equivalente a h5read)
        data = f[data_path][:]

except Exception as e:
    raise RuntimeError(f"It's not possible to read file attributes. Error: {e}")

# Convert to double precision (float64), equivalent to MATLAB double()
X = X.astype(np.float64)

#acq = das.read(filename    )
#data = acq.data
#dx = acq.dx
#fs = acq.fs
#num_locs = acq.channel_number

Reading data...
Metadata is successful:
 -> fs = 20000.0 Hz
 -> dx = 0.532 m
 -> num_locs = 556 Channels


In [23]:
'''
Vectors
'''
read_locs, num_tr = X.shape
if read_locs != num_locs:
    #warnings.warn(
    #    f"Number of loci read ({read_locs}) is different from metadata ({num_locs})!"
    #)
    num_locs = read_locs
t = np.arange(num_tr) / fs
y = np.arange(num_locs).reshape(-1, 1) * dx
'''
Before Processing
Temporal Cut
'''
t_start_cut = 0.0
t_end_cut = 15.0

'''
Saving the original values for the feedback
'''
t_original_end = t[-1]
num_tr_original = num_tr

'''
Finding the start and end index from the time vector
'''
idx_start = np.where(t >= t_start_cut)[0]
idx_start = idx_start[0] if idx_start.size > 0 else None

idx_end = np.where(t <= t_end_cut)[0]
idx_end = idx_end[-1] if idx_end.size > 0 else None


In [24]:
if (idx_start is None or idx_end is None or idx_end <= idx_start):
    raise ValueError(
        f"Invalid time interval [{t_start_cut:.2f} s, {t_end_cut:.2f} s] "
        f"or outside limits [0, {t_original_end:.2f} s]."
    )

In [25]:
'''
Perform temporal cut on DATA matrix (columns) and TIME vector ---
'''
X = X[:, idx_start:idx_end + 1]
t = t[idx_start:idx_end + 1]

'''
Update dependent variables
'''
num_tr = X.shape[1]

print("Temporal cut completed.")
print(f" -> Original time: 0.00 s to {t_original_end:.2f} s ({num_tr_original} samples)")
print(f" -> New time:      {t[0]:.2f} s to {t[-1]:.2f} s ({num_tr} samples)")

Temporal cut completed.
 -> Original time: 0.00 s to 0.03 s (556 samples)
 -> New time:      0.00 s to 0.03 s (556 samples)


In [26]:
print('Removing the DC mean from the channel...')
X = X - np.mean(X, axis=1, keepdims=True)

Removing the DC mean from the channel...


In [27]:
'''
Pre-Processing parameters
'''
hp_cut = 0.1
lp_cut = 9000

startFiber = 30
endFiber = 200

startFiberperfil = 120
endFiberperfil = 150

scale_factor = 0.02292

phasestart = -2
phaseend = 2

'''
Butterworth band-pass
'''
sos = butter(
    N=4,
    Wn=[hp_cut, lp_cut],
    btype="bandpass",
    fs=fs,
    output="sos"
)

X = sosfiltfilt(sos, X, axis=1)

In [28]:
'''
Plot
'''

tmap = len(t) / fs
nshow = min(num_tr, int(round(tmap * fs)))

print("Finding closest visualization channels...")

chan_view = np.argmin(np.abs(y - startFiberperfil))
chan_view2 = np.argmin(np.abs(y - endFiberperfil))

Finding closest visualization channels...


In [46]:
def _as_scalar(x, name="value"):
    """
    Safely convert input to a scalar.

    Parameters:
    -----------
    x : array-like
        Input value to convert to scalar
    name : str, optional
        Variable name for error messages

    Returns:
    --------
    scalar
        Scalar value

    Raises:
    -------
    ValueError
        If input cannot be converted to scalar
    """
    try:
        x = np.asarray(x)
        if x.ndim == 0:
            return x.item()
        if x.size == 1:
            return x.ravel()[0].item()
        raise ValueError(f"{name} must be a scalar (or size-1 array), got shape={x.shape}")
    except Exception as e:
        raise ValueError(f"Failed to convert {name} to scalar: {e}")

def validate_inputs():
    """Validate all required global variables exist and have correct shapes."""
    required_vars = ['t', 'fs', 'num_tr', 'y', 'X', 'phasestart', 'phaseend',
                    'startFiberperfil', 'endFiberperfil', 'hp_cut', 'lp_cut',
                    'num_locs', 'startFiber', 'endFiber', 'scale_factor']

    for var in required_vars:
        if var not in globals():
            raise NameError(f"Required variable '{var}' is not defined")

    # Check array shapes consistency
    if X.shape[0] != len(y):
        raise ValueError(f"Mismatch between X shape {X.shape} and y length {len(y)}")
    if X.shape[1] != len(t):
        raise ValueError(f"Mismatch between X shape {X.shape} and t length {len(t)}")